In [1]:
%pip install --upgrade pysymphony
%pip install python-dotenv


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Setting the environment

In [2]:
import os
import getpass

if "SYMPHONY_API_KEY" not in os.environ:
    os.environ["SYMPHONY_API_KEY"] = getpass.getpass("Enter your Symphony API key: ")

## Initializing the client

In [3]:
from pysymphony import SymphonyClient
import logging

logging.basicConfig(level=logging.INFO)

client = SymphonyClient()

INFO:root:GET https://inference.composition-labs.com/inference/tools/list: RUNNING
INFO:root:GET https://inference.composition-labs.com/inference/tools/list: SUCCESS


## Check available tools

In [4]:
client.tools

[{'id': 'pplx_online',
  'name': 'Perplexity Online',
  'description': 'Useful for getting internet enabled (access to internet) natural language answers to input queries without citations.',
  'type': <ToolType.SYMPHONY: 'symphony'>},
 {'id': 'openai_img_to_text',
  'name': 'OpenAI Image to Text',
  'description': 'Useful for analyzing and describing images, detecting objects, and extracting text from images.',
  'type': <ToolType.SYMPHONY: 'symphony'>},
 {'id': 'clinical_trials_gov_data_retriever',
  'name': 'Clinical Trials Gov Data Retriever',
  'description': 'Useful for retrieving a list of clinical studies for the pharma and biotech indistries, based on a given query, or details of a single clinical study based on a given NCT number. Pass studies_query to get a list of studies based on the query. Pass nct_id to get details of a single study. Never pass both studies_query and nct_id.',
  'type': <ToolType.SYMPHONY: 'symphony'>},
 {'id': 'developer_cloudflare_search_tool',
  'name

## Adding a local tool

In [5]:
from pysymphony.settings import settings

def add_tool(a: int, b: int):
    """
    Add two numbers
    """
    print(f"`add_tool` tool called on the client side with inputs {a} and {b}")
    return a + b

settings.add_local_tool(add_tool)

In [6]:
client.tools

[{'id': 'pplx_online',
  'name': 'Perplexity Online',
  'description': 'Useful for getting internet enabled (access to internet) natural language answers to input queries without citations.',
  'type': <ToolType.SYMPHONY: 'symphony'>},
 {'id': 'openai_img_to_text',
  'name': 'OpenAI Image to Text',
  'description': 'Useful for analyzing and describing images, detecting objects, and extracting text from images.',
  'type': <ToolType.SYMPHONY: 'symphony'>},
 {'id': 'clinical_trials_gov_data_retriever',
  'name': 'Clinical Trials Gov Data Retriever',
  'description': 'Useful for retrieving a list of clinical studies for the pharma and biotech indistries, based on a given query, or details of a single clinical study based on a given NCT number. Pass studies_query to get a list of studies based on the query. Pass nct_id to get details of a single study. Never pass both studies_query and nct_id.',
  'type': <ToolType.SYMPHONY: 'symphony'>},
 {'id': 'developer_cloudflare_search_tool',
  'name

## Generating a workflow

In [7]:
from pysymphony.workflows import Workflow

task = (
    "Given a list of numbers, return the sum of the numbers."
)
parameters = {
    # total number of agents required (set to something)
    "max_steps": 1
}
# list of tools to use
useful_tools = ["add_tool"]
workflow = client.generate_workflow(task, parameters, useful_tools)
print(workflow)

INFO:root:POST https://inference.composition-labs.com/inference/planner/generateworkflow: SUCCESS


Workflow(id=825d91f5-f895-4e9f-976f-1f3838cc2a09, task_description=Given a list of numbers, return the sum of the numbers.)


## Visualizing the workflow

In [8]:
from IPython.display import Image, display
display(Image(url=workflow.get_mermaid_image_url()))

## Running the workflow

In [9]:
# initialize the run
run = workflow.init_run(input="12312413312312, 12312312312, 12312312, 123123")

INFO:root:POST https://inference.composition-labs.com/inference/runner/initrun: SUCCESS


In [10]:
# run the workflow (use the `run_step` method to run steps iteratively)
run.run()

INFO:root:POST https://inference.composition-labs.com/inference/runner/runstep: SUCCESS


`add_tool` tool called on the client side with inputs 0 and 12312413312312


INFO:root:POST https://inference.composition-labs.com/inference/runner/runstep: SUCCESS


`add_tool` tool called on the client side with inputs 12312413312312 and 12312312312


INFO:root:POST https://inference.composition-labs.com/inference/runner/runstep: SUCCESS


`add_tool` tool called on the client side with inputs 12324725624624 and 12312312


INFO:root:POST https://inference.composition-labs.com/inference/runner/runstep: SUCCESS


`add_tool` tool called on the client side with inputs 12324737936936 and 123123


INFO:root:POST https://inference.composition-labs.com/inference/runner/runstep: SUCCESS


<Status.SUCCESS: 'success'>

## Displaying the trace

In [11]:
# display the inputs and outputs of each step
run.ios

{'0': {'input': None,
  'output': '12312413312312, 12312312312, 12312312, 123123'},
 '1': {'input': 'List of numbers: 12312413312312, 12312312312, 12312312, 123123',
  'output': 'The total sum of the numbers is 12324738060059.'}}

In [12]:
# display the output of a step
from IPython.display import Markdown
Markdown(run.ios["1"]['output'])

The total sum of the numbers is 12324738060059.

## Generating a workflow with multiple steps

In [23]:
from pysymphony.workflows import Workflow

task = (
    "Given a company name, find the competitors, recent news, company history, recent financials, and the founder's background and output a json of the information."
)
parameters = {
    "max_steps": 6
}
useful_tools = ["pplx_online"]
workflow = client.generate_workflow(task, parameters, useful_tools)

INFO:root:POST https://inference.composition-labs.com/inference/planner/generateworkflow: SUCCESS


In [24]:
display(Image(url=workflow.get_mermaid_image_url()))

In [25]:
run = workflow.init_run(input="Apple")
run.run()
run.ios

INFO:root:POST https://inference.composition-labs.com/inference/runner/initrun: SUCCESS
INFO:root:POST https://inference.composition-labs.com/inference/runner/runstep: SUCCESS
INFO:root:POST https://inference.composition-labs.com/inference/runner/runstep: SUCCESS
INFO:root:POST https://inference.composition-labs.com/inference/runner/runstep: SUCCESS
INFO:root:POST https://inference.composition-labs.com/inference/runner/runstep: SUCCESS


{'0': {'input': None, 'output': 'Apple'},
 '1': {'input': 'Company name: Apple',
  'output': 'Here is a list of key competitors for Apple across various product categories:\n\n### Smartphones\n1. **Samsung**\n2. **Huawei**\n3. **Xiaomi**\n4. **Oppo**\n\n### Tablets\n1. **Samsung (Galaxy Tab series)**\n2. **Huawei (MatePad series)**\n\n### Smartwatches\n1. **Samsung (Galaxy Watch)**\n2. **Huawei (Watch GT series)**\n\n### Laptops\n1. **Huawei (MateBook series)**\n2. **Dell**\n3. **HP**\n4. **Lenovo**\n\n### Operating Systems\n1. **Android (used by various manufacturers)**\n\n### Services\n1. **Samsung Pay**\n2. **Bixby**\n3. **Huawei Services**\n\n### Additional Competitors\n1. **Arista Networks**\n2. **Dell Technologies**\n3. **HP**\n4. **Hewlett Packard Enterprise**\n5. **Super Micro Computer**\n6. **Seagate Technology**\n7. **Pure Storage**\n8. **Logitech International**\n9. **Omnicell**\n10. **NCR Voyix**\n\nThese competitors span multiple segments of the tech industry, reflecting t

In [27]:
from IPython.display import Markdown
Markdown(run.ios["6"]['output'])

Here is the structured JSON format containing the compiled information about Apple:

```json
{
  "competitors": {
    "smartphones": [
      "Samsung",
      "Huawei",
      "Xiaomi",
      "Oppo"
    ],
    "tablets": [
      "Samsung (Galaxy Tab series)",
      "Huawei (MatePad series)"
    ],
    "smartwatches": [
      "Samsung (Galaxy Watch)",
      "Huawei (Watch GT series)"
    ],
    "laptops": [
      "Huawei (MateBook series)",
      "Dell",
      "HP",
      "Lenovo"
    ],
    "operating_systems": [
      "Android (used by various manufacturers)"
    ],
    "services": [
      "Samsung Pay",
      "Bixby",
      "Huawei Services"
    ],
    "additional_competitors": [
      "Arista Networks",
      "Dell Technologies",
      "HP",
      "Hewlett Packard Enterprise",
      "Super Micro Computer",
      "Seagate Technology",
      "Pure Storage",
      "Logitech International",
      "Omnicell",
      "NCR Voyix"
    ]
  },
  "recent_news": [
    {
      "title": "Apple Reaches First-Ever Union Contract",
      "source": "NBC News",
      "summary": "Apple Inc. has reached its first-ever union contract with store employees in Maryland, marking a significant milestone in labor relations for the tech giant.",
      "date": "September 2024"
    },
    {
      "title": "iPhone 16 Launch",
      "source": "Apple Newsroom",
      "summary": "Apple has launched the iPhone 16 lineup, along with the AirPods 4 and Apple Watch Series 10, globally.",
      "date": "September 20, 2024"
    },
    {
      "title": "Software Updates",
      "source": "Apple Newsroom",
      "summary": "Apple has released several software updates, including iOS 18, macOS Sequoia, iPadOS 18, watchOS 11, and visionOS 2 for Apple Vision Pro.",
      "date": "September 16, 2024"
    },
    {
      "title": "Apple Wallet Integration",
      "source": "Apple Newsroom",
      "summary": "Apple has brought California driver’s licenses and state IDs to Apple Wallet, enhancing the functionality of the service.",
      "date": "September 19, 2024"
    },
    {
      "title": "New Health Features",
      "source": "Apple Press Release",
      "summary": "Apple has introduced groundbreaking health features, including those available with the new AirPods Pro 2, focusing on hearing health.",
      "date": "September 9, 2024"
    },
    {
      "title": "Apple Intelligence",
      "source": "Apple Press Release",
      "summary": "Apple Intelligence will be coming to iPhone, iPad, and Mac starting next month, enhancing the security and functionality of these devices.",
      "date": "September 9, 2024"
    },
    {
      "title": "Apple Arcade Updates",
      "source": "Apple Newsroom",
      "summary": "Eight new games have joined Apple Arcade, including Balatro+, expanding the gaming options available to users.",
      "date": "September 5, 2024"
    },
    {
      "title": "Apple Sports Ready for Football Season",
      "source": "Apple Newsroom",
      "summary": "Apple Sports is ready for the football season, providing comprehensive coverage and updates for sports enthusiasts.",
      "date": "August 29, 2024"
    }
  ],
  "company_history": {
    "founding": {
      "date": "April 1, 1976",
      "founders": ["Steve Jobs", "Steve Wozniak", "Ronald Wayne"],
      "initial_product": "Apple I"
    },
    "early_years": {
      "incorporation": {
        "date": "January 3, 1977",
        "name": "Apple Computer, Inc."
      },
      "major_product": "Apple II",
      "revenue_growth": {
        "1977": "$775,000",
        "1980": "$118 million"
      }
    },
    "expansion_and_challenges": {
      "public_offering": {
        "date": "1980",
        "shares_sold": "4.6 million",
        "initial_price": "$22"
      },
      "macintosh_launch": {
        "date": "1984",
        "features": ["graphical user interface", "mouse"]
      },
      "leadership_changes": {
        "1985": "Steve Jobs left Apple",
        "1997": "Jobs returned after acquiring NeXT"
      },
      "financial_struggles": {
        "period": "1990s",
        "status": "near bankruptcy"
      }
    },
    "resurgence_and_innovation": {
      "imac_launch": {
        "date": "1998",
        "significance": "shift in design philosophy"
      },
      "ipod_launch": {
        "date": "2001",
        "impact": "revolutionized the music industry"
      },
      "itunes_store_launch": {
        "date": "2003",
        "significance": "platform for digital music sales"
      }
    },
    "diversification_and_global_dominance": {
      "name_change": {
        "date": "January 9, 2007",
        "new_name": "Apple Inc."
      },
      "iphone_launch": {
        "date": "2007",
        "impact": "transformed the smartphone industry"
      },
      "ipad_launch": {
        "date": "2010",
        "significance": "expanded product line"
      },
      "leadership_transition": {
        "date": "2011",
        "new_ceo": "Tim Cook"
      }
    },
    "recent_developments": {
      "apple_watch_launch": {
        "date": "2015",
        "significance": "expanded wearable technology offerings"
      },
      "market_value": {
        "date": "2018",
        "value": "$1 trillion"
      },
      "apple_vision_pro_announcement": {
        "date": "2023",
        "significance": "step in augmented reality technology"
      }
    }
  },
  "recent_financials": {
    "company": "Apple",
    "financials": {
      "recent_quarter": {
        "revenue": "85.8 billion",
        "net_income": "21.45 billion",
        "gross_profit": "39.68 billion",
        "operating_cash_flow": "28.86 billion"
      },
      "year_to_date": {
        "revenue": "296.1 billion",
        "net_income": "79 billion"
      },
      "services_revenue": "new all-time high"
    },
    "stock_performance": {
      "summary": "Strong financial results contributing to overall market value."
    }
  },
  "founder_background": {
    "name": "Steven Paul Jobs",
    "birth": "February 24, 1955, in San Francisco, California",
    "adoption": "Adopted by Clara and Paul Jobs.",
    "educational_background": {
      "Reed_College": "Enrolled in 1972 but dropped out after one semester, continuing to attend classes that interested him, including calligraphy.",
      "De_Anza_College": "Attended over two semesters in the 1970s."
    },
    "previous_ventures": [
      {
        "company": "Apple Inc.",
        "description": "Co-founded in 1976 with Steve Wozniak; launched the Apple I and Apple II."
      },
      {
        "company": "NeXT Inc.",
        "description": "Founded in 1985 after leaving Apple; focused on a specialized operating system."
      },
      {
        "company": "Pixar Animation Studios",
        "description": "Acquired in 1986; produced iconic films like 'Toy Story.'"
      }
    ],
    "notable_achievements": [
      "Revitalizing Apple: Returned in 1997, leading to the launch of innovative products like the iMac, iPod, and iPhone.",
      "Design Influence: His calligraphy studies influenced Apple's design philosophy.",
      "Awards: Received the Vollum Award for Distinguished Accomplishment in Science and Technology in 1991.",
      "Legacy: Remembered for his visionary leadership and significant impact on the technology industry."
    ]
  }
}
``` 

This JSON structure organizes the information clearly and concisely, making it easy to access and understand the various aspects of Apple's business, history, and founder.